In [1]:
!pip install -q transformers accelerate bitsandbytes peft datasets trl sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 28.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 105.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 35.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 66.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import json
from datasets import Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from transformers import TrainingArguments, Trainer

2025-11-14 10:26:57.517343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763116017.701200      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763116017.749783      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
# -------------------------
# Config / paths (edit if needed)
# -------------------------
MODEL_NAME = "mistralai/Mistral-7B-v0.1"
DATA_PATH = "/kaggle/input/glaive-coding-assistant/c9bc9129-eba0-4b10-8292-4ae70fc7fa0d.json"  # replace with your file
OUTPUT_DIR = "generator_lora_25k"
NUM_SAMPLES = 5000         # <-- reduce to 12k for fast training <1hr
NUM_EPOCHS = 1
MAX_LENGTH = 128            # shorter sequences, big speed win
PER_DEVICE_BATCH = 4
GRAD_ACCUM = 4  

In [4]:
# -------------------------
# Load dataset and format prompts
# -------------------------
def load_qa_dataset(path):
    with open(path, "r", encoding="utf-8") as f:
        raw = json.load(f)

    samples = []
    for row in raw:
        q = row.get("question", "").strip()
        a = row.get("answer", "").strip()
        if not q or not a:
            continue

        # no trailing newline after "Answer:" to save tokens
        prompt = f"Question: {q}\nAnswer:"
        completion = a
        samples.append({"prompt": prompt, "completion": completion})

    return Dataset.from_list(samples)

dataset = load_qa_dataset(DATA_PATH)
print("Loaded samples:", len(dataset))
print("Example sample:", dataset[0])

# reduce dataset to NUM_SAMPLES (shuffle first)
if len(dataset) > NUM_SAMPLES:
    dataset = dataset.shuffle(seed=42).select(range(NUM_SAMPLES))
print("Using samples:", len(dataset))

Loaded samples: 136108
Example sample: {'prompt': 'Question: How can I output bold text in Bash? I have a Bash script that prints some text to the screen using the `echo "Some Text"` command. Is there a way I can format the text to make it bold?\nAnswer:', 'completion': 'Yes, you can format the output text in Bash to make it bold. Bash allows you to use special escape sequences for text decoration. To make some text bold in bash, you would use the escape sequence `\\033`, and to reset the formatting, you would use `\\033`. \n\nHere\'s how you can update your `echo` statement to print bold text:\n\n```bash\necho -e "\\033Some Text\\033"\n```\n\nIn this code:\n\n- The `-e` option of `echo` allows the interpretation of backslash escapes.\n- The `\\033` sequence sets the text to be bold.\n- The `Some Text` part is the actual text that will be printed in bold.\n- The `\\033` sequence resets the text formatting to the default, so anything printed afterwards will be in the default format.\n\n

In [5]:
# -------------------------
# Load tokenizer + model (4-bit)
# -------------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# fast tokenizer (fallback to use_fast=False if issues)
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
except Exception:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Attempting to load 4-bit base model (will try FlashAttention2 then fallback)...")

def load_base_model():
    # try with flash_attention_2 first (if supported by the HF code for the model)
    try:
        return AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            quantization_config=bnb_config,
            device_map="auto",
            attn_implementation="flash_attention_2",
            trust_remote_code=True,
        )
    except Exception:
        # fallback
        return AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True,
        )

base_model = load_base_model()

# prepare base for k-bit training then apply LoRA
base_model = prepare_model_for_kbit_training(base_model)


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Attempting to load 4-bit base model (will try FlashAttention2 then fallback)...


...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [6]:
# -------------------------
# LoRA config (replace)
# -------------------------
lora_cfg = LoraConfig(
    r=4,                          # smaller rank -> faster
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"],  # minimal/fast, good for Q/A
)


model = get_peft_model(base_model, lora_cfg)
model.print_trainable_parameters()

trainable params: 1,703,936 || all params: 7,243,436,032 || trainable%: 0.0235


In [7]:
# -------------------------
# Tokenize function (batched for speed)
# -------------------------
def tokenize_function(examples):
    prompts = examples["prompt"]
    completions = examples["completion"]
    full_texts = [p + c + (tokenizer.eos_token or "") for p, c in zip(prompts, completions)]

    # tokenize prompts and full texts (batched) to get prompt lengths and full tokens
    tokenized_full = tokenizer(
        full_texts,
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False,
    )
    tokenized_prompts = tokenizer(
        prompts,
        truncation=True,
        max_length=MAX_LENGTH,
        padding=False,
        add_special_tokens=False,
    )

    labels = []
    for full_ids, prompt_ids in zip(tokenized_full["input_ids"], tokenized_prompts["input_ids"]):
        lab = full_ids.copy()
        prompt_len = len(prompt_ids)
        # mask prompt tokens (loss only on completion)
        for i in range(min(prompt_len, len(lab))):
            lab[i] = -100
        labels.append(lab)

    return {
        "input_ids": tokenized_full["input_ids"],
        "attention_mask": tokenized_full["attention_mask"],
        "labels": labels,
    }

# batched mapping (faster)
tokenized_ds = dataset.map(tokenize_function, batched=True, batch_size=2000, remove_columns=dataset.column_names)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [8]:
# -------------------------
# Collate function for Trainer
# -------------------------
def collate_fn(batch):
    input_ids = [torch.tensor(x["input_ids"], dtype=torch.long) for x in batch]
    labels = [torch.tensor(x["labels"], dtype=torch.long) for x in batch]

    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)

    attention_mask = (input_ids != tokenizer.pad_token_id).long()

    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}


In [9]:
# -------------------------
# TrainingArguments + Trainer (optimized)
# -------------------------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=PER_DEVICE_BATCH,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=2e-4,
    fp16=True,
    bf16=False,
    num_train_epochs=NUM_EPOCHS,
    save_strategy="epoch",
    save_total_limit=2,
    logging_steps=50,
    remove_unused_columns=False,
    report_to="none",
    dataloader_num_workers=4,    # speed up data loading
)
import torch
torch.backends.cudnn.benchmark = True
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    data_collator=collate_fn,
    tokenizer=tokenizer,
)

/tmp/ipykernel_48/1001498512.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
# -------------------------
# Train
# -------------------------
trainer.train()

# Save the LoRA adapter (and optionally a small config)
model.save_pretrained(OUTPUT_DIR)
print("✅ LoRA adapter saved in:", OUTPUT_DIR)

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabli

Step,Training Loss
50,0.570100
100,0.537800
150,0.514500
200,0.503100
250,0.517600
300,0.509800


✅ LoRA adapter saved in: generator_lora_25k


In [13]:
# -------------------------
# Loading for generation (example)
# -------------------------
# reload base model (4-bit) and attach adapter for inference
base = load_base_model()
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

gen_model = PeftModel.from_pretrained(base, OUTPUT_DIR)
gen_model.eval()



ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [19]:
# Example generation function that returns only the newly generated answer text
def generate_answer(question, max_new_tokens=200, **gen_kwargs):
    prompt = f"Question: {question}\nAnswer:"  # no trailing newline
    inputs = tokenizer(prompt, return_tensors="pt").to(gen_model.device)

    generated = gen_model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=gen_kwargs.get("do_sample", True),
        temperature=gen_kwargs.get("temperature", 0.8),
        top_p=gen_kwargs.get("top_p", 0.95),
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    decoded = tokenizer.decode(generated[0], skip_special_tokens=True)
    if decoded.startswith(prompt):
        return decoded[len(prompt):].strip()
    if "Answer:" in decoded:
        return decoded.split("Answer:", 1)[1].strip()
    return decoded.strip()

# Quick test
q = "Can someone help me write a MATLAB program to generate a random array of 5 elements?"
print("Generated answer:\n", generate_answer(q, max_new_tokens=400))

Generated answer:
 Certainly! Here's a MATLAB program that generates a random array of 5 elements:

```matlab
% Generate random array of 5 elements
n = 5;

% Initialize a random number generator
rng('default');

% Generate random numbers
a = rand(1,n);
```

This program initializes a random number generator with the `rng` function using the default seed. It then uses the `rand` function to generate a random array of 5 elements. The `rand` function returns a single column vector of random numbers between 0 and 1. By passing the desired number of elements (5) to the `rand` function, we create a 1x5 array containing 5 random numbers.

The generated array, `a`, will contain 5 unique random numbers between 0 and 1. If you need random integers instead, you can multiply the generated numbers by the desired maximum value and round them using the `round` function:

```matlab
% Generate random integers
b = round(a * 100);
```

In this example, `b` will contain 5 random integers between 0 and 99.

In [15]:
!zip -r output.zip /kaggle/working/generator_lora_25k

  adding: kaggle/working/generator_lora_25k/ (stored 0%)
  adding: kaggle/working/generator_lora_25k/README.md (deflated 66%)
  adding: kaggle/working/generator_lora_25k/adapter_config.json (deflated 55%)
  adding: kaggle/working/generator_lora_25k/checkpoint-313/ (stored 0%)
  adding: kaggle/working/generator_lora_25k/checkpoint-313/README.md (deflated 66%)
  adding: kaggle/working/generator_lora_25k/checkpoint-313/scaler.pt (deflated 60%)
  adding: kaggle/working/generator_lora_25k/checkpoint-313/optimizer.pt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 8%)
  adding: kaggle/working/generator_lora_25k/checkpoint-313/special_tokens_map.json (deflated 73%)
  adding: kaggle/working/generator_lora_25k/checkpoint-313/tokenizer.json (deflated 85%)
  adding: kaggle/working/generator_lora_25k/checkpoint-313/tokenizer.model (deflated 55%)
  adding: kaggle/working/generator_lora_25k/checkpoint-313/adapter_config.json (deflated 55%)
  adding: kaggle/working/generator_lora_25k/checkpoint-313/tokenizer_config.json (deflated 68%)
  adding: kaggle/working/generator_lora_25k/checkpoint-313/training_args.bin (deflated 51%)
  adding: kaggle/working/generator_lora_25k/checkpoint-313/adapter_model.safetensors (deflated 7%)
  adding: kaggle/working/generator_lora_25k/checkpoint-313/scheduler.pt (deflated 56%)
  adding: kaggle/working/generator_lora_25k/checkpoint-313/trainer_state.json (deflated 65%)
  adding: kaggle/working/generator_lora_25k/checkpoint-313/rng_state.pth (deflated 25%)
  adding: kaggle/working/generator_lora_25k/adapter_model.s

In [17]:
from IPython.display import FileLink
from IPython.display import display

In [18]:
display(FileLink('output.zip'))

/kaggle/working/output.zip